# Praktikum am Astropeiler Stockert

![Image](https://farm6.staticflickr.com/5561/14499025000_c58c919221_z.jpg)

## Imports

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.style import use
use('ggplot')
%matplotlib inline
%config InlineBackend.figure_format = 'svg'
plt.rcParams['figure.figsize'] = (9, 6)
plt.rcParams['font.size'] = 12
plt.rcParams['legend.fontsize'] = 12
plt.rcParams['lines.linewidth'] = 2

from IPython.display import display, Math, Latex

def pprint(*args, **kwargs):
    display(Latex(*args, **kwargs))

In [ ]:
sidereal_day_seconds = 86164.09054
hydrogen_line_ghz = 1.42040575177

In [ ]:
from scipy.constants import c as speed_of_light

def time2degrees(timestamp, declination):
    '''transforms timestamp into degrees in the field of view'''
    t = timestamp - timestamp.min()
    t = t.seconds + t.microseconds/1e6 + t.nanoseconds/1e9
    degrees = 360 / sidereal_day_seconds * t * np.cos(declination)
    return degrees

def frequency2velocity(f_obs, f_emit):
    ratio = f_emit / f_obs
    return (ratio**2 - 1) / (ratio**2 + 1) * speed_of_light

def gauss_uniform(x, N, mu, sigma, U):
    # assure norm and sigma positive:
    if N < 0 or sigma < 0:
        return np.nan
    norm = N / np.sqrt(2 * np.pi * sigma**2) 
    return norm * np.exp(-0.5 * (x - mu)**2/sigma**2) + U

## Resolution Measurement



In [ ]:
data = pd.read_csv(
    'data/sto25_CAS_A_cont_10781.csv',
    header=None,
    names=['time',
           'azimuth',
           'elevation',
           'right_ascension',
           'declination',
           'phase',
           'intensity_v',
           'intensity_h',
          ],
    index_col=0,
    parse_dates=[0],
)


data.head()

In [ ]:
data['degrees'] = time2degrees(data.index, data.declination.mean()/180 * np.pi)

In [ ]:
from scipy.optimize import curve_fit

fit_data = data.query('degrees > 1 & degrees < 3.5')

params1, cov1 = curve_fit(f=gauss_uniform,
                          xdata=fit_data.degrees,
                          ydata=fit_data.intensity_v,
                          p0=[1e6, 4, 2, 2e6],
                          )
params2, cov2 = curve_fit(f=gauss_uniform,
                          xdata=fit_data.degrees,
                          ydata=fit_data.intensity_h,
                          p0=[1e6, 4, 2, 2e6],
                          )

sigma1 = params1[2]
fwhm1 = 2*np.sqrt(2*np.log(2)) * sigma1
sigma2 = params2[2]
fwhm2 = 2*np.sqrt(2*np.log(2)) * sigma2

pprint('$\sigma_1 = {:1.2f}^\circ$'.format(sigma1))
pprint('$\mathrm{{FWHM}}_1 = {:1.2f}^\circ$'.format(fwhm1))
pprint('$\sigma_2 = {:1.2f}^\circ$'.format(sigma2))
pprint('$\mathrm{{FWHM}}_2 = {:1.2f}^\circ$'.format(fwhm2))

In [ ]:
px = np.linspace(0, 7.5, 1000)
data.plot(x='degrees', y=['intensity_v', 'intensity_h'])
plt.plot(px, gauss_uniform(px, *params1), label='fit1', ls='--')
plt.plot(px, gauss_uniform(px, *params2), label='fit2', ls='--')
plt.legend()
plt.show()

## OH-measurement

In [ ]:
oh_on = pd.read_csv('data/sto25_W3OH_spec_10782.csv',
                    sep=' ',
                    skiprows=44,
                    header=None,
                    names=['index',
                           'frequency',
                           'data1',
                           'data2',
                           'data3',
                           'data4',
                           'data_sum',
                          ]
                   )
oh_off = pd.read_csv('data/sto25_W3OH_spec_10783.csv',
                     sep=' ',
                     skiprows=44,
                     header=None,
                     names=['index',
                            'frequency',
                            'data1',
                            'data2',
                            'data3',
                            'data4',
                            'data_sum',
                           ]
                    )


# transform frequency to GHz
oh_on.frequency /= 1e9
oh_off.frequency /= 1e9

oh_on = oh_on.query('frequency > 1.65 & frequency < 1.68')
oh_off = oh_off.query('frequency > 1.65 & frequency < 1.68')

In [ ]:
plt.plot(oh_on.frequency, oh_on.data_sum, label='on')
plt.plot(oh_off.frequency, oh_off.data_sum, label='off')
plt.legend()
plt.xlabel('frequency / GHz')
plt.ylabel('Intensity / a.u.')

In [ ]:
plt.plot(oh_on.frequency, oh_on.data_sum - oh_off.data_sum, label='on - off')
plt.legend()
plt.xlabel('frequency / MHz')
plt.ylabel('Intensity / a.u.')

# Extragalatic HI Measurement – Holmberg II

In [ ]:
holmberg_on = pd.read_csv(
    'data/sto25_UGC4305_spec_10784.csv',
    sep=' ',
    skiprows=44,
    header=None,
    names=['index',
           'frequency',
           'data1',
           'data2',
           'data3',
           'data4',
           'data_sum',
          ],
)

holmberg_off = pd.read_csv(
    'data/sto25_UGC4305_spec_10785.csv',
    sep=' ',
    skiprows=44,
    header=None,
    names=['index',
           'frequency',
           'data1',
           'data2',
           'data3',
           'data4',
           'data_sum',
          ],
)

# transform frequency to GHz
holmberg_on.frequency /= 1e9
holmberg_off.frequency /= 1e9

In [ ]:
holmberg = holmberg_on[['frequency', 'data_sum']].copy()
holmberg.columns = ['frequency', 'data_on']

holmberg['data_off'] = holmberg_off.data_sum
holmberg['data_diff'] = holmberg.data_on - holmberg.data_off 
holmberg['velocity'] = frequency2velocity(holmberg.frequency, hydrogen_line_ghz) / 1000

holmberg = holmberg.query('frequency <= 1.422 & frequency >= 1.418')

In [ ]:
holmberg.plot(x='velocity', y=['data_on', 'data_off'])
plt.xlabel('velocity / (km/s)')

In [ ]:
holmberg.plot(x='velocity', y='data_diff', label='on - off')
plt.xlabel('velocity / (km/s)')